## YouTube and Code Reference - Nicholas Renotte


https://www.youtube.com/watch?v=Mut_u40Sqz4&list=PLgNJO2hghbmjlE6cuKMws2ejC54BTAaWV&index=8
    
https://github.com/nicknochnack/ReinforcementLearningCourse/blob/main/Main%20Course.ipynb    

### 1. Import dependencies

In [1]:
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os

### 2. Load Environment

In [2]:
environment_name = "CartPole-v0"
env = gym.make(environment_name)

In [3]:
#Test - variables
act=env.action_space.sample()
act

1

In [4]:
env.action_space

Discrete(2)

In [5]:
state = env.reset()
env.step(act)

(array([ 0.02419967,  0.14883744, -0.03858816, -0.31942153]), 1.0, False, {})

In [6]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [7]:
env.observation_space.sample()

array([-8.3619463e-01, -2.9396347e+38,  3.9190453e-01, -1.2681797e+38],
      dtype=float32)

In [8]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:15.0
Episode:2 Score:11.0
Episode:3 Score:13.0
Episode:4 Score:15.0
Episode:5 Score:42.0


### Understanding The Environment

In [16]:
#https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [9]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [10]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([-1.0343683e+00,  2.2732914e+38,  3.2578310e-01, -1.8233075e+38],
      dtype=float32)

### 3. Train an RL Model

In [11]:
log_path = os.path.join('Training', 'logs')
log_path

'Training\\logs'

In [12]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
#model = PPO('MlpPolicy', env, verbose = 1,tensorbaord_log=log_path)
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)


Using cpu device


In [13]:
model.learn(total_timesteps=20000)

Logging to Training\logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 403  |
|    iterations      | 1    |
|    time_elapsed    | 5    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 541         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008976722 |
|    clip_fraction        | 0.0907      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00241    |
|    learning_rate        | 0.0003      |
|    loss                 | 5.6         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0124     |
|    value_loss           | 48.9        |
-----------------------------------------
---

### 4. Save and Reload Model

In [14]:
PPO_path = os.path.join('Training', 'PPO_model')

In [15]:
model.save(PPO_path)

In [17]:
del model

In [18]:
model = PPO.load('Training\PPO_model', env=env)

### 5. Evaluation

In [16]:
from stable_baselines3.common.evaluation import evaluate_policy

In [17]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\shilp\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [18]:
env.close()

### 6. Test Model

In [19]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([ 0.0085816 ,  0.32481961,  0.11396407, -0.06890463])}]


In [20]:
env.close()

### 7. Viewing Logs in Tensorboard

In [21]:

training_log_path = os.path.join(log_path, 'PPO_3')

In [62]:
# Execute on cmd prompt -- !tensorboard --logdir={'Training/logs/PPO'} 
# OR
# go to cmd prompt cd Training/logs/PPO and type !tensorboard --logdir=.

### 8. Adding a callback to the training Stage

In [ ]:
# Stop training once reward threshold is reached

In [22]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [23]:
save_path = os.path.join('Training')
log_path = os.path.join('Training', 'logs')

In [24]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [25]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [26]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [27]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 1284 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1012       |
|    iterations           | 2          |
|    time_elapsed         | 4          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00711492 |
|    clip_fraction        | 0.0777     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.687     |
|    explained_variance   | 0.00294    |
|    learning_rate        | 0.0003     |
|    loss                 | 8.53       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.011     |
|    value_loss           | 50.9       |
----------------------------------------
---------------------

In [28]:
model_path = os.path.join('Training', 'best_model')
model = PPO.load(model_path, env=env)

In [29]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [30]:
env.close()

### 9. Changing Policies

In [31]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})
model.learn(total_timesteps=20000, callback=eval_callback)

Using cpu device
-----------------------------
| time/              |      |
|    fps             | 1345 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 786         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015035283 |
|    clip_fraction        | 0.223       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.00219    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.47        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0213     |
|    value_loss           | 17.7        |
-----------------------------------------
-----------------

### 9. Using an Alternate Algorithm

In [32]:
from stable_baselines3 import DQN

In [33]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)
model.learn(total_timesteps=20000, callback=eval_callback)

Using cpu device
Logging to Training\logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.937    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 4899     |
|    time_elapsed     | 0        |
|    total_timesteps  | 132      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.898    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 4728     |
|    time_elapsed     | 0        |
|    total_timesteps  | 215      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.866    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 4886     |
|    time_elapsed     | 0        |
|    total_timesteps  | 282      |
----------------------------------
-------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 6394     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2451     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 6356     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2522     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 6354     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2604     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 6326     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4998     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 6310     |
|    time_elapsed     | 0        |
|    total_timesteps  | 5054     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 6239     |
|    time_elapsed     | 0        |
|    total_timesteps  | 5159     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 6215     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7245     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 6159     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7328     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 6148     |
|    time_elapsed     | 1        |
|    total_timesteps  | 7453     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 424      |
|    fps              | 6192     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9600     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 428      |
|    fps              | 6181     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9656     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 6194     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9731     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 6262     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11707    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 6282     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11800    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 6297     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11880    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 640      |
|    fps              | 6497     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14119    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 6502     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14220    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 6498     |
|    time_elapsed     | 2        |
|    total_timesteps  | 14278    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 748      |
|    fps              | 6457     |
|    time_elapsed     | 2        |
|    total_timesteps  | 16582    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 6452     |
|    time_elapsed     | 2        |
|    total_timesteps  | 16646    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 6449     |
|    time_elapsed     | 2        |
|    total_timesteps  | 16729    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 856      |
|    fps              | 6472     |
|    time_elapsed     | 2        |
|    total_timesteps  | 18905    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 6476     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19014    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 6482     |
|    time_elapsed     | 2        |
|    total_timesteps  | 19086    |
----------------------------------
----------------------------------
| rollout/          

In [35]:
dqn_path = os.path.join('Training',  'DQN_model')
model.save(dqn_path)

In [36]:
model = DQN.load(dqn_path, env=env)

In [37]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(9.0, 0.6324555320336759)

In [38]:
env.close()